# Анализ результатов эксперимента от 30.09.25

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_df(ax, df, tags=None, title="Training curves"):
    for tag in tags:
        sns.lineplot(data=df, x="step", y=tag, ax=ax, label=tag)
    
    ax.set_title(title, fontsize=14)
    ax.set_xlabel("Step")
    ax.set_ylabel("Value")
    ax.legend(title="Metric")

## Описание эксперимента

Обучение нейронной сети с учетом всех нароботок, включая предварительный сбор данных.

## Результаты

In [ ]:
from pathlib import Path

EXPERIMENT_NAME = "td3_train_real_async"
EXPERIMENT_DATE = "2025-09-30"
EXPERIMENT_TIME = "13-08-38"

PATH_TO_EXP_DIR = Path(f"../experiments/{EXPERIMENT_NAME}/{EXPERIMENT_DATE}_{EXPERIMENT_TIME}")

ENV_LOG_DIR = PATH_TO_EXP_DIR / "env_logs"
TRAIN_LOG_DIR = PATH_TO_EXP_DIR / "train_logs"

In [ ]:
import re
import pandas as pd

log_pattern_interaction = re.compile(
    r"step=(?P<step>\d+)"
    r" time=(?P<time>\d+\.\d+)"
    r" phase=(?P<phase>\w+)"
    r" block_step=(?P<block_step>\d+)"
    r" kp=(?P<kp>-?\d+\.\d+)"
    r" ki=(?P<ki>-?\d+\.\d+)"
    r" kd=(?P<kd>-?\d+\.\d+)"
    r" process_variable=(?P<process_variable>-?\d+\.\d+)"
    r" control_output=(?P<control_output>-?\d+\.\d+)"
    r" u_min=(?P<u_min>-?\d+\.\d+)"
    r" u_max=(?P<u_max>-?\d+\.\d+)"
)

log_pattern_model = re.compile(
    r"step=(?P<step>\d+)"
    r" phase=(?P<phase>\w+)"
    r" block_step=(?P<block_step>final)"
    r" kp=(?P<kp>-?\d+\.\d+)"
    r" ki=(?P<ki>-?\d+\.\d+)"
    r" kd=(?P<kd>-?\d+\.\d+)"
    r" error_mean=(?P<error_mean>-?\d+\.\d+)"
    r" error_std=(?P<error_std>-?\d+\.\d+)"
    r" error_mean_norm=(?P<error_mean_norm>-?\d+\.\d+)"
    r" error_std_norm=(?P<error_std_norm>-?\d+\.\d+)"
    r" reward=(?P<reward>-?\d+\.\d+)"
)

def parse_logs_split(file_path: str):
    interaction, model = [], []
    with open(file_path, "r") as f:
        for line in f:
            line = line.strip()
            if line.startswith("step"):
                m1 = log_pattern_interaction.match(line)
                if m1:
                    interaction.append(m1.groupdict())
                    continue
                m2 = log_pattern_model.match(line)
                if m2:
                    model.append(m2.groupdict())
                    continue
    interaction_df = pd.DataFrame(interaction)
    model_df = pd.DataFrame(model)

    for df in (interaction_df, model_df):
        if not df.empty:
            df["step"] = df["step"].astype(int)
            for col in df.columns:
                if col != "phase" and col != "block_step":
                    df[col] = pd.to_numeric(df[col], errors="coerce")

    return interaction_df, model_df


interaction_df, model_df = parse_logs_split(ENV_LOG_DIR / "log.txt")

In [ ]:
print("=== Interaction logs info ===")
print(interaction_df.info())
print("\n=== Interaction logs head ===")
print(interaction_df.head())
print("\n=== Interaction logs describe ===")
print(interaction_df.describe())

## Графики

In [ ]:
#phase = "warmup"
phase = "pretrain"
#phase = "normal"

In [ ]:
filtered_interaction_df = interaction_df[interaction_df["phase"] == phase].copy()

fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

sns.lineplot(data=filtered_interaction_df, x="step", y="process_variable", ax=axes[0], color="blue")
axes[0].set_title("Process Variable")
axes[0].set_ylabel("process_variable")

sns.lineplot(data=filtered_interaction_df, x="step", y="control_output", ax=axes[1], color="red")
axes[1].set_title("Control Output")
axes[1].set_xlabel("Step")
axes[1].set_ylabel("control_output")

plt.tight_layout()
plt.savefig(f'process_controll_sigma_{phase}.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
print("=== Model logs info ===")
print(model_df.info())
print("\n=== Model logs head ===")
print(model_df.head())
print("\n=== Model logs describe ===")
print(model_df.describe())

Фильтрация по фазе тренировки.

In [ ]:
#phase = "warmup"
phase = "pretrain"
#phase = "normal"


from enum import Enum

NUM_STEPS_IN_BLOCK = 200


class Phase(Enum):
    WARMUP = "warmup" 
    PRETRAIN = "pretrain" # исследование со случайными коэффициентами
    NORMAL = "normal" # работа модели

target_phase = Phase.PRETRAIN

filtered_df = model_df[model_df["phase"] == target_phase.value].copy()
filtered_df["block"] = filtered_df["step"] // NUM_STEPS_IN_BLOCK

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 10), sharex=True)

sns.lineplot(data=filtered_df[2400:2450], x="block", y="kp", ax=axes[0], color="blue")
axes[0].set_title("kp over Steps")
axes[0].set_ylabel("kp")

sns.lineplot(data=filtered_df[2400:2450], x="block", y="ki", ax=axes[1], color="orange")
axes[1].set_title("ki over Steps")
axes[1].set_ylabel("ki")

sns.lineplot(data=filtered_df[2400:2450], x="block", y="kd", ax=axes[2], color="green")
axes[2].set_title("kd over Steps")
axes[2].set_xlabel("Block Step")
axes[2].set_ylabel("kd")

plt.tight_layout()
plt.savefig(f'coefficients_sigma_{phase}.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

sns.lineplot(data=filtered_df[2400:2450], x="block", y="error_mean", ax=axes[0], color="blue")
axes[0].set_title("Error Mean (Raw)")
axes[0].set_ylabel("error_mean")

sns.lineplot(data=filtered_df[2400:2450], x="block", y="error_mean_norm", ax=axes[1], color="red")
axes[1].set_title("Error Mean (Normalized)")
axes[1].set_xlabel("Block Step")
axes[1].set_ylabel("error_mean_norm")

plt.tight_layout()
plt.savefig(f'error_mean_sigma_{phase}.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

sns.lineplot(data=filtered_df[2400:2450], x="block", y="error_std", ax=axes[0], color="blue")
axes[0].set_title("Error STD (Raw)")
axes[0].set_ylabel("error_std")

sns.lineplot(data=filtered_df[2400:2450], x="block", y="error_std_norm", ax=axes[1], color="red")
axes[1].set_title("Error STD (Normalized)")
axes[1].set_xlabel("Block Step")
axes[1].set_ylabel("error_std_norm")

plt.tight_layout()
plt.savefig(f'error_std_sigma_{phase}.png', dpi=300, bbox_inches='tight')
plt.show()

## График для ВКВО

In [ ]:
from matplotlib.ticker import MultipleLocator

plt.rcParams.update({
    'font.size': 14,
    'axes.labelsize': 16,
    'axes.titlesize': 18,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'legend.fontsize': 14
})

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

ax1.plot(filtered_df["block"], filtered_df["error_std"], color="green", label="Среднеквадратичное отклонение ошибки")
ax1.set_ylabel("СКВО ошибки", color="green")
ax1.tick_params(axis='y', labelcolor="green")
ax1.set_xlim(2400, 2450)
ax1.set_ylim(0, 200)
ax1.legend(loc="best")
ax1.xaxis.set_minor_locator(MultipleLocator(5))
ax1.grid(True, axis='x', which='both', linestyle='--', alpha=0.4)

ax2.plot(filtered_df["block"], filtered_df["kp"], color="red", label=r"Коэффициент $K_{\text{p}}$")
ax2.set_xlabel("Время (с)")
ax2.set_ylabel(r"$K_{\text{p}}$", color="red")
ax2.tick_params(axis='y', labelcolor="red")
ax2.set_ylim(4, 13)
ax2.legend(loc="best")
ax2.xaxis.set_minor_locator(MultipleLocator(5))
ax2.grid(True, axis='x', which='both', linestyle='--', alpha=0.4)

axes[0].plot(filtered_interaction_df["time_sec"], filtered_interaction_df["process_variable"], color="blue")
axes[0].set_ylabel("Напряжение на фотодетекторе (АЦП", color='blue')
axes[0].set_ylim(1000, 1500)
axes[0].tick_params(axis='y', labelcolor="blue")
axes[0].xaxis.set_minor_locator(MultipleLocator(5))
axes[0].grid(True, axis='x', which='both', linestyle='--', alpha=0.4)

plt.tight_layout()
# plt.savefig("free-run_split.png", dpi=300)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import MultipleLocator

plt.rcParams.update({
    'font.size': 14,
    'axes.labelsize': 16,
    'axes.titlesize': 18,
    'xtick.labelsize': 14,
    'ytick.labelsize': 14,
    'legend.fontsize': 14
})

# --- Создаём фигуру и сетку: 2 строки × 2 столбца ---
fig = plt.figure(figsize=(20, 8))
gs = GridSpec(2, 2, figure=fig, width_ratios=[1, 1], height_ratios=[1, 1])

# --- Левый верхний график ---
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(filtered_df["block"], filtered_df["error_std"], color="red", label="Среднеквадратичное отклонение ошибки")
ax1.set_ylabel("СКВО ошибки", color="red")
ax1.tick_params(axis='y', labelcolor="red")
ax1.set_xlim(2400, 2450)
ax1.set_ylim(0, 200)
ax1.legend(loc="best")
ax1.xaxis.set_minor_locator(MultipleLocator(5))
ax1.grid(True, axis='x', which='both', linestyle='--', alpha=0.4)

# --- Левый нижний график ---
ax2 = fig.add_subplot(gs[1, 0], sharex=ax1)
ax2.plot(filtered_df["block"], filtered_df["kp"], color="blue", label=r"Коэффициент $K_{\text{p}}$")
ax2.set_xlabel("Время (с)")
ax2.set_ylabel(r"$K_{\text{p}}$", color="blue")
ax2.tick_params(axis='y', labelcolor="blue")
ax2.set_ylim(4, 13)
ax2.legend(loc="best")
ax2.xaxis.set_minor_locator(MultipleLocator(5))
ax2.grid(True, axis='x', which='both', linestyle='--', alpha=0.4)

SETPOINT = 1200

# --- Правый большой график (занимает обе строки) ---
ax3 = fig.add_subplot(gs[:, 1])  # обе строки второго столбца
ax3.plot(filtered_interaction_df["time_sec"], filtered_interaction_df["process_variable"], color="green", label=r"$U_{\text{АЦП}}$")
ax3.set_ylabel("Напряжение на фотодетекторе (АЦП)", color='green')
ax3.axhline(SETPOINT, color="black", linestyle="--")
ax3.set_xlabel("Время (с)")
ax3.set_xlim(2400, 2450)
ax3.set_ylim(1000, 1500)
ax3.legend(loc='best')
ax3.tick_params(axis='y', labelcolor="green")
ax3.xaxis.set_minor_locator(MultipleLocator(5))
ax3.grid(True, axis='x', which='both', linestyle='--', alpha=0.4)

plt.tight_layout()
plt.savefig("kp_split_combo.png", dpi=300, bbox_inches="tight")
plt.show()

In [ ]:
corr_kp_std = filtered_df["kp"].corr(filtered_df["error_std"])

print(f"Коэффициент корреляции между Kp и std ошибки: {corr_kp_std:.4f}")

In [ ]:
mask = (filtered_df["block"] >= 2400) & (filtered_df["block"] <= 2450)
subset = filtered_df.loc[mask]

corr_kp_std = subset["kp"].corr(subset["error_std"])

print(f"Коэффициент корреляции между Kp и std ошибки (блоки 2400–2450): {corr_kp_std:.4f}")

In [ ]:
filtered_interaction_df = filtered_interaction_df.copy()
filtered_interaction_df["time_sec"] = filtered_interaction_df["step"] * 0.005  # 5 мс = 0.005 с

fig, axes = plt.subplots(2, 1, figsize=(10, 8), sharex=True)

axes[0].plot(filtered_interaction_df["time_sec"], filtered_interaction_df["process_variable"], color="blue")
axes[0].set_ylabel("Напряжение на фотодетекторе (АЦП", color='blue')
axes[0].set_ylim(1000, 1500)
axes[0].tick_params(axis='y', labelcolor="blue")
axes[0].xaxis.set_minor_locator(MultipleLocator(5))
axes[0].grid(True, axis='x', which='both', linestyle='--', alpha=0.4)

axes[1].plot(filtered_interaction_df["time_sec"], filtered_interaction_df["control_output"], color="red")
axes[1].set_title("Control Output")
axes[1].set_xlabel("Время (с)")
axes[1].set_ylabel("control_output")
axes[1].set_ylim(1750, 2750)
axes[1].grid(True, axis='y', linestyle='--', alpha=0.5)
axes[1].xaxis.set_minor_locator(MultipleLocator(5))
axes[1].grid(True, axis='x', which='both', linestyle='--', alpha=0.4)

axes[0].set_xlim(2400, 2450)

plt.tight_layout()
# plt.savefig(f'process_controll_sigma_{phase}.png', dpi=300, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
sns.lineplot(data=filtered_df[2400:2450], x="block", y="reward", ax=ax, color="green")
ax.set_title("Reward over Steps")
ax.set_xlabel("Block Step")
ax.set_ylabel("Reward")
plt.savefig(f'error_reward_sigma_{phase}.png', dpi=300, bbox_inches='tight')
plt.show()

## Статистика

In [ ]:
target_phase = Phase.PRETRAIN

filtered_model_df = model_df[model_df["phase"] == target_phase.value].copy()

print("=== Model logs info (phase == 'pretrain') ===")
print(filtered_model_df.info())
print("\n=== Model logs head (phase == 'pretrain') ===")
print(filtered_model_df.head())
print("\n=== Model logs describe (phase == 'pretrain') ===")
print(filtered_model_df.describe())

Новые коэффициенты для нормализации будут 250 для mean и 200 для std. 

## Ход обучения

In [ ]:
log_pattern_rl = re.compile(
    r"step=(?P<step>\d+)\s+Loss/Critic=(?P<critic_loss>-?\d+\.\d+)"
    r"\s+Loss/Actor=(?P<actor_loss>-?\d+\.\d+)"
)

def parse_rl_logs(file_path: str):
    rows = []
    with open(file_path, "r") as f:
        for line in f:
            line = line.strip()
            m = log_pattern_rl.match(line)
            if m:
                rows.append({
                    "step": int(m.group("step")),
                    "critic_loss": float(m.group("critic_loss")),
                    "actor_loss": float(m.group("actor_loss")),
                })
    df = pd.DataFrame(rows)
    return df

rl_logs_df = parse_rl_logs(TRAIN_LOG_DIR / "log.txt") 

In [ ]:
print("RL training logs info:")
print(rl_logs_df.info())
print("\nRL training logs head:")
print(rl_logs_df.head())
print("\nRL training logs describe:")
print(rl_logs_df.describe())

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
plot_df(ax, rl_logs_df, tags=["critic_loss", "actor_loss"], title="Training loss")
plt.savefig(f'learning_{phase}.png', dpi=300, bbox_inches='tight')